In [483]:
#import numpy as np
import pandas as pd
from itertools import combinations
#from collections import Counter
#import regex as re
#import seaborn as sns
#import matplotlib.pyplot as plt

In [484]:
data_path = '/content/sample_data/movie_bd_v5.csv'
data = pd.read_csv(data_path)
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1784,tt0484740,45000000,4607608,Love in the Time of Cholera,Javier Bardem|Unax Ugalde|Benjamin Bratt|Catal...,Mike Newell,How long would you wait for love?,"In Colombia just after the Great War, an old m...",139,Drama|Romance,New Line Cinema|Grosvenor Park Media Ltd.,10/4/2007,6.6,2007
1074,tt1621045,12000000,96070507,Think Like a Man,Michael Ealy|Jerry Ferrara|Meagan Good|Regina ...,Tim Story,Let the mind games begin,Four friends conspire to turn the tables on th...,123,Comedy|Romance,Rainforest Films|Screen Gems|Studio Mir,4/16/2012,6.6,2012
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
207,tt1100089,24000000,12096300,Foxcatcher,Steve Carell|Channing Tatum|Mark Ruffalo|Sienn...,Bennett Miller,Based on the shocking true story,The greatest Olympic Wrestling Champion brothe...,134,Drama|Thriller,Likely Story|Media Rights Capital|Annapurna Pi...,5/19/2014,6.5,2014
947,tt0265662,20000000,75597042,The Rookie,Dennis Quaid|Rachel Griffiths|Beth Grant|Angus...,John Lee Hancock,Sometimes dreams come back to life.,Jim Morris never made it out of the minor leag...,127,Drama|Family,Walt Disney Pictures|Gran Via Productions|98 M...,3/25/2002,6.0,2002


In [485]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [486]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [487]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:
#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

'''Для наглядности все добавленные столбцы буду наименовать с символа '_'.
   Изначально менял столбцы на месте и потом пришлось всё переписать, 
   но для академических целей так будет лучше и удобней.'''

"Для наглядности все добавленные столбцы буду наименовать с символа '_'.\n   Изначально менял столбцы на месте и потом пришлось всё переписать, \n   но для академических целей так будет лучше и удобней."

In [488]:
# Столбец profit (прибыль) поможет отвечать на вопросы [6:]
data['_profit']=data.revenue-data.budget

In [489]:
# Дату выхода переведём формат времени'''
data['_release_date'] = pd.to_datetime(data.release_date,infer_datetime_format=True)

In [490]:
# Для удобства работы со столбцами содержащими несколько значений
# добавим столбцы со списками значений вместо строк
# Опеределим список стобцов содержащих строки с символом '|'
columns_to_split = data.applymap(lambda x:str(x).find('|')+1).sum()
columns_to_split = columns_to_split[columns_to_split!=0].index.to_list()
# Заменим строки в найденых столбцах на списки значений
for column in columns_to_split:
  data['_'+column] = data[column].str.split('|')#.apply(sorted)
# При решении задания 27 наткнулся на необходимость обязательно сортировать
# А именно в фильме 374 актёры перечислены в порядке, оличном от
# фильмов [242, 497, 722, 864, 1312, 1560, 1670] и без предварительной
# сортировки один фильм выпадает из подсчёта

In [491]:
# Для демонстрации другого подхода определим все возможные жанры
genres = sorted(list(set('|'.join(set(data.genres)).split('|'))))
_genres = ['_'+x for x in genres]

# %timeit data.genres.str.split('|').explode().unique().tolist() 7.8 ms per loop
# %timeit pd.Series(data.genres.unique()).str.split('|').explode().unique().tolist() 3.53 ms per loop
# %timeit pd.Series('|'.join(data.genres.unique().tolist()).split('|')).unique().tolist() 847 µs per loop
# %timeit list(set('|'.join(set(data.genres)).split('|'))) 434 µs per loop

# Дополним датафрейм столбцами-флагами об отнесении фильма к конкретному жанру
for item in genres:
  data['_'+item] =  data.genres.apply(lambda x: --(item in x))

In [492]:
# Для ответа на вопрос 24
data['_title_lenght'] = data.original_title.str.len()

In [493]:
# Для ответа на вопрос 25
data['_overview_word_count'] = data.overview.str.split('\W+').apply(len)

In [494]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   imdb_id                1889 non-null   object        
 1   budget                 1889 non-null   int64         
 2   revenue                1889 non-null   int64         
 3   original_title         1889 non-null   object        
 4   cast                   1889 non-null   object        
 5   director               1889 non-null   object        
 6   tagline                1889 non-null   object        
 7   overview               1889 non-null   object        
 8   runtime                1889 non-null   int64         
 9   genres                 1889 non-null   object        
 10  production_companies   1889 non-null   object        
 11  release_date           1889 non-null   object        
 12  vote_average           1889 non-null   float64       
 13  rel

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [495]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [496]:
# тут пишем ваш код для решения данного вопроса:
'''Функция для форматирования строки ответа как в примере'''
def film_to_str(item):
  return '{}. {} ({})' \
    .format(str(item.name), item.original_title, str(item.imdb_id))

'''Определим наибольший бюджет среди всех фильмов.'''
max_budget = data.budget.max()
'''Выберем строку по совпадению значения бюджета с наибольшим бюджетом.
   Если их несколько - возьмём первый по порядку в датафрейме.'''
selected = data.query('budget==@max_budget').iloc[0]

'''Сформируем ответ как в примере лаконично за счёт .iloc[] и f''.'''
answers['1'] = film_to_str(selected)

ВАРИАНТ 2

In [497]:
# можно добавлять разные варианты решения
'''Перечислю прочие запомнившиеся методы поиска и обращения'''
selected = data[data.budget==data.describe()['budget']['max']]
selected = data[data['budget']==data.budget.max()]
selected = data.sort_values(by='budget',ascending=False).iloc[0]
selected = data.iloc[data['budget'].idxmax()]
selected = data.nlargest(1,'budget')

# 2. Какой из фильмов самый длительный (в минутах)?

In [498]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '1157. Gods and Generals (tt0279111)'
'''Да простит меня проверяющий, далее не стал отдельно в комментариях
   указывать найденные варианты ответов. Их поиск важнее и интересней.
   А все ответы разом можно посмотреть в конце.'''

'Да простит меня проверяющий, далее не стал отдельно в комментариях\n   указывать найденные варианты ответов. Их поиск важнее и интересней.\n   А все ответы разом можно посмотреть в конце.'

In [499]:
# Поступим по аналогии с поиском решения на первый вопрос
max_runtime = data.runtime.max()
selected = data.query('runtime==@max_runtime').iloc[0]
answers['2'] = film_to_str(selected)

# 3. Какой из фильмов самый короткий (в минутах)?





In [500]:
# Поступим по аналогии с поиском решения на первый вопрос
min_runtime = data.runtime.min()
selected = data.query('runtime==@min_runtime').iloc[0]
answers['3'] = film_to_str(selected)

# 4. Какова средняя длительность фильмов?


In [501]:
# Возьмём отдельно столбец длительность и изучим
answers['4'] = round(data.runtime.mean())

# 5. Каково медианное значение длительности фильмов? 

In [502]:
# Здесь можно не округлять, так как значение из набора
answers['5'] = round(data.runtime.median())

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [503]:
# лучше код получения столбца profit вынести в Предобработку что в начале
'''Рекомендация выполнена. Остальное по аналогии с первым вопросом'''
max_profit = data._profit.max()
selected = data.query('_profit==@max_profit').iloc[0]
answers['6'] = film_to_str(selected)

# 7. Какой фильм самый убыточный? 

In [504]:
# Поступим по аналогии с поиском решения на первый вопрос
min_profit = data._profit.min()
selected = data.query('_profit==@min_profit').iloc[0]
answers['7'] = film_to_str(selected)

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [505]:
# Фильтруем и считаем уникальные значения
answers['8'] = data.query('_profit>0').imdb_id.nunique()

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [506]:
# Поступим по аналогии с поиском решения на первый вопрос
selected = data.query('release_year==2008') \
.sort_values(by='_profit',ascending=False).iloc[0]
answers['9'] = film_to_str(selected)

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [507]:
# Поступим по аналогии с поиском решения на первый вопрос
selected = data.query('2012<=release_year<=2014') \
  .sort_values(by='_profit').iloc[0]
answers['10'] = film_to_str(selected)

# 11. Какого жанра фильмов больше всего?

In [508]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
'''Благодаря столбцам-флагам определяем количества простой суммой'''
answers['11'] = data[_genres].sum().nlargest(1).index[0][1:]

ВАРИАНТ 2

In [509]:
'''Вариант без необходимости дополнения столбцами-флагами
   для случая режиссёров и актёров добавление столбцов неэкономно'''
display(data.explode('_genres').groupby('_genres')['imdb_id'].count() \
        .nlargest(1).index[0])

'''Вариант через value_counts без группировки и агрегации'''
display(data.explode('_genres')._genres.value_counts().nlargest(1).index[0])

'''Вариант решения через обход датафрейма с составлением словаря'''
genres_dict = {genre:0 for genre in genres}
for row in data.index:
  for genre in data.iloc[row]._genres:
    genres_dict[genre]+=1
'''Собранный словарь с количествами проанализируем удобным способом'''
display(pd.Series(genres_dict).nlargest(1).index[0])

'''Самый короткий по длинне вариант, притом без необходимости в предобработке'''
Counter(data.genres.apply(str.split,sep='|').explode()).most_common()[0][0]

'Drama'

'Drama'

'Drama'

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [510]:
# Фильтруем, аггрегируем, находим больший
answers['12'] = data.query('_profit>0')[_genres].sum().nlargest(1).index[0][1:]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [511]:
# Разворачиваем, группируем, агрегируем, находим больший
answers['13'] = data.explode(column='_director').groupby('_director') \
  .revenue.sum().nlargest(1).index[0]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [512]:
# Разворачиваем, группируем, агрегируем, находим больший
answers['14'] = data.explode(column='_director').groupby('_director') \
  ._Action.sum().nlargest(1).index[0]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [513]:
# Фильтруем, разворачиваем, группируем, агрегируем, находим больший
answers['15'] = data.query('release_year==2012').explode(column='_cast') \
  .groupby('_cast').revenue.sum().nlargest(1).index[0]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [514]:
# Фильтруем, разворачиваем, группируем, агрегируем, находим больший
answers['16'] = data[data.budget>data.budget.mean()].explode(column='_cast') \
  .groupby('_cast').imdb_id.count().nlargest(1).index[0]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [515]:
# Разворачиваем актёров и фильтруем по нужному
# Разворачиваем жанры и группируем по ним
# Агрегируем количество фильмов и находим большее
answers['17'] = data.explode(column='_cast').query('_cast=="Nicolas Cage"') \
  .explode('_genres').groupby('_genres').imdb_id \
  .count().nlargest(1).index[0]

# Вариант через дополнительные столбцы
# Разворачиваем актёров и фильтруем по нужному
# Отбираем столбцы жанров, суммируем и находим больший
data.explode(column='_cast').query('_cast=="Nicolas Cage"')[_genres].sum() \
  .nlargest(1).index[0][1:]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [516]:
# Фильтруем, находим ниаменьший по прибыли
# Отличается от предыдущих так как не было шага группировки по искмому значению
selected = data[data.production_companies.str.contains("Paramount Pictures")] \
  .nsmallest(1,'_profit').iloc[0]
answers['18'] = film_to_str(selected)

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [517]:
# Группируем, агрегируем, находим больший
answers['19'] = data.groupby('release_year').revenue.sum().nlargest(1).index[0]

# 20. Какой самый прибыльный год для студии Warner Bros?

In [518]:
# Фильтруем, группируем, агрегируем, находим больший
selected = data.production_companies.str.contains("Warner Bros")
answers['20'] = data[selected].groupby('release_year') \
  ._profit.sum().nlargest(1).index[0]

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [519]:
# Фильтруем, группируем, агрегируем, находим больший
answers['21'] = data.groupby(by=data._release_date.dt.month_name()) \
  .imdb_id.count().nlargest(1).index[0]

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [520]:
# Фильтруем и считаем уникальные значения
selected = (data._release_date.dt.month == 6) \
          |(data._release_date.dt.month == 7) \
          |(data._release_date.dt.month == 8)
answers['22'] = data[selected].imdb_id.nunique()

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [521]:
# Фильтруем, группируем, агрегируем, находим больший
selected = (data._release_date.dt.month == 12) \
          |(data._release_date.dt.month == 1 ) \
          |(data._release_date.dt.month == 2 )
answers['23'] = data[selected].explode('_director').groupby('_director') \
  .imdb_id.count().nlargest(1).index[0]

In [522]:
# Подсмотрел на Слаке, но ещё не освоился
data[selected].explode('_director')._director.value_counts().index[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные **названия** своим фильмам по количеству символов?

In [523]:
# Фильтруем, группируем, агрегируем, находим больший
answers['24'] = data.explode('_production_companies') \
  .groupby('_production_companies')._title_lenght.mean().nlargest(1).index[0]

# 25. **Описание** фильмов какой студии в среднем самые длинные по количеству слов?

In [524]:
# Список сравниваемых компаний
to_check = [ \
  'Universal Pictures', \
  'Warner Bros', \
  'Midnight Picture Show', \
  'Paramount Pictures', \
  'Total Entertainment' \
]

In [525]:
# Функция для группировки по списку, что-бы считать компании
# ['Warner Bros.', 
#  'Warner Bros. Animation', 
#  'Warner Bros. Interactive Entertainment', 
#  'Warner Bros. Pictures']
# как один вариант 'Warner Bros'
def check(item):
  for company in to_check:
    if str(item).find(company)>=0:
      return company
  return 'None'

# Развернём компании, установим в качестве индекса, сгруппируем по функции,
# агрегируем, сортируем и выводим найденную компанию
answers['25'] = data.explode('_production_companies') \
  [['_production_companies','_overview_word_count']] \
  .set_index('_production_companies').groupby(check) \
  .mean().sort_values(by='_overview_word_count',ascending=False).index[0]

In [526]:
# Вариант 2
# Проход по списку с подсчётом среднего количества слов для каждой компании
result = {}
for item in to_check:
  result.setdefault(item,0)
  result[item]=data[data.production_companies.str.contains(item)]._overview_word_count.mean()
result

{'Midnight Picture Show': 177.0,
 'Paramount Pictures': 57.778688524590166,
 'Total Entertainment': 88.0,
 'Universal Pictures': 56.38857142857143,
 'Warner Bros': 56.67213114754098}

In [527]:
'''Остатки мук поиска ошибки в решении и самого решения'''
print(len(data[data.production_companies.str.contains('Total Entertainment')].original_title.str.split().tolist()[0]))
len(data[data.production_companies.str.contains('Total Entertainment')].original_title.str.split('\W+').tolist()[0])
#selected = data.production_companies.str.contains('.*Universal Pictures.*|.*Warner Bros.*|.*Midnight Picture Show.*|.*Paramount Pictures.*|.*Total Entertainment.*',regex=True)

#data.explode('_production_companies').groupby(check)._overview_word_count.mean()
#data[selected].explode('_production_companies').groupby(data[selected].explode('_production_companies')['_production_companies'].apply(check)).mean()

#answers['25'] = data.explode('_production_companies')[['_production_companies','_overview_word_count']].set_index('_production_companies').groupby(check).mean().sort_values(by='_overview_word_count',ascending=False).index[0]
#data.explode('_production_companies')[data.explode('_production_companies')['_production_companies'].str.contains('Warner Bros')]['_production_companies'].unique()
#data[data['production_companies'].str.contains('Midnight Picture Show')]
#data.explode('_production_companies')['_production_companies'].apply(check)

#data.explode('_production_companies')[data.explode('_production_companies')['_production_companies'].apply(check)].groupby('_production_companies')._overview_word_count \
#  .mean()
  #.nlargest(1).index[0]

# Разворачиваем, группируем, агрегируем, находим больший
#answers['25'] = data.explode('_production_companies') \
#  .groupby('_production_companies')._overview_word_count \
#  .mean().nlargest(1).index[0]
#df = data[['production_companies','_production_companies','original_title','_overview_word_count']].sort_values(by='_overview_word_count', ascending=False)
#df.explode('_production_companies').groupby('_production_companies').mean().sort_values(by='_overview_word_count',ascending=False)
#df['production_companies'].str.contains()

8


7

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [528]:
# Список искомых комбинаций
to_check = [ \
  ['Inside Out', 'The Dark Knight', '12 Years a Slave'], \
  ['BloodRayne', 'The Adventures of Rocky & Bullwinkle'], \
  ['Batman Begins', \
   'The Lord of the Rings: The Return of the King', \
   'Upside Down' \
  ], \
  ['300', 'Lucky Number Slevin', 'Kill Bill: Vol. 1'], \
  ['Upside Down', 'Inside Out', 'Iron Man'] \
]

# Фильтруем по 1% квантилю
selected = data['vote_average']>data['vote_average'].quantile(0.99)

for list_of_titles in to_check:
  if set(list_of_titles).issubset(data[selected].original_title.tolist()):
      answers['26'] = list_of_titles

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [529]:
# Список искомых комбинаций
to_check = [ \
  ('Johnny Depp','Helena Bonham Carter'), \
  ('Ben Stiller','Owen Wilson'), \
  ('Vin Diesel','Paul Walker'), \
  ('Adam Sandler','Kevin James'), \
  ('Daniel Radcliffe','Rupert Grint') \
]
# Упорядочим
to_check = list(map(tuple,map(sorted,to_check)))

# Сортируем списки актёров для детерминированности составляемых комбинаций
# Создаём парные комбинации актёров из списка актёров фильма
# Материализуем их, разворачиваем, считаем, сравниваем с искомым списком
# Сортируем по убыванию частоты съёмки пары в фильме и вуаля
answers['27'] = data['_cast'].apply(sorted).apply(combinations,r=2) \
  .apply(list).explode().value_counts().loc[to_check] \
  .sort_values(ascending=False).index[0]

In [530]:
'''Остатки мук поиска причины расхождения двух способов подсчёта
   секрет кроется в фильме 374 из за которого теряется одна комбинация
   "Emma Watson|Rupert Grint", так как без предварительной сортировки 
   считает вариант "Rupert Grint|Emma Watson" в количестве 7 штук,
   а с предварительной сортировкой "Emma Watson|Rupert Grint" - 8 штук'''
#top1 = 
selected = data['_cast'].apply(combinations,r=2).apply(list).explode().value_counts() \
.reset_index()#.apply(sorted,columns='index')#.groupby(level=0).count().nlargest(5)
selected['index'] = selected['index'].apply(sorted).apply('|'.join)
selected[selected['index'].str.contains('Emma Watson')]
data[data.cast.str.contains('Emma Watson') & data.cast.str.contains('Rupert Grint')][['original_title','_cast']]
#selected.groupby('index').count().nlargest(5,'_cast')
#selected = data['_cast'].apply(combinations,r=2).apply(list).explode().value_counts().head(5).index.tolist()
#temp = {}
#for item in selected:
#  data['recount'] = data[data.cast.str.contains(item[0]) \
#     & data.cast.str.contains(item[1])].imdb_id.nunique()
#selected
#top2 = data['_cast'].apply(sorted).apply(combinations,r=2).apply(list).explode().value_counts().head(5)
#Counter(data['_cast'].apply(combinations,r=2).apply(list).explode()).most_common(5)
#data['_cast'].apply(combinations,r=2).apply(list).explode().reset_index().groupby('_cast')._cast.count().nlargest(5)

#data[data.cast.str.contains('Daniel Radcliffe') \
#& data.cast.str.contains('Emma Watson')]
#selected = []
#for pair in top1.append(top2).index:
#  selected.append(data[data.cast.str.contains(pair[0]) & data.cast.str.contains(pair[1])].index.tolist())
#selected = pd.Series(selected).explode().unique().tolist()
#data[['original_title','_cast']].loc[selected]['_cast'].apply(sorted).apply(combinations,r=2).apply(list).explode().value_counts().head(5)
#data._cast = data.cast.str.split('|').apply(sorted)
#data._cast
#data['_cast'].apply(combinations,r=2).apply(list).apply(lambda x:sorted(x))
#data[data.cast.str.contains('Rupert Grint') \
#& data.cast.str.contains('Emma Watson')].info()

,original_title,_cast
242,Harry Potter and the Half-Blood Prince,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ..."
374,Harry Potter and the Deathly Hallows: Part 1,"[Daniel Radcliffe, Emma Watson, Rupert Grint, ..."
497,Harry Potter and the Philosopher's Stone,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ..."
722,Harry Potter and the Deathly Hallows: Part 2,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ..."
864,Harry Potter and the Chamber of Secrets,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ..."
1312,Harry Potter and the Goblet of Fire,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ..."
1560,Harry Potter and the Prisoner of Azkaban,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ..."
1670,Harry Potter and the Order of the Phoenix,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ..."


ВАРИАНТ 2

In [531]:
# Применение объекта Counter позволяет написать короче, но меньше гибкости
display(Counter(data['_cast'].apply(sorted).apply(combinations,r=2) \
                .apply(list).explode()).most_common()[:5])
print()

# Тут типа мускулатурой играю - хвастаюсь насколько тему понял
[data['_cast'].apply(sorted).apply(combinations,r=x).apply(list).explode() \
  .value_counts().nlargest(1,keep='all').to_dict() for x in range(2,6)]

[(('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Emma Watson', 'Rupert Grint'), 8),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Helena Bonham Carter', 'Johnny Depp'), 6)]

[{('Daniel Radcliffe', 'Emma Watson'): 8,
  ('Daniel Radcliffe', 'Rupert Grint'): 8,
  ('Emma Watson', 'Rupert Grint'): 8},
 {('Daniel Radcliffe', 'Emma Watson', 'Rupert Grint'): 8},
 {('Jennifer Lawrence',
   'Josh Hutcherson',
   'Liam Hemsworth',
   'Woody Harrelson'): 4,
  ('Kristen Stewart',
   'Peter Facinelli',
   'Robert Pattinson',
   'Taylor Lautner'): 4},
 {('Antonio Banderas',
   'Cameron Diaz',
   'Eddie Murphy',
   'Julie Andrews',
   'Mike Myers'): 3,
  ('Elijah Wood',
   'Ian McKellen',
   'Liv Tyler',
   'Orlando Bloom',
   'Viggo Mortensen'): 3,
  ('Elizabeth Banks',
   'Jennifer Lawrence',
   'Josh Hutcherson',
   'Liam Hemsworth',
   'Woody Harrelson'): 3,
  ('Famke Janssen',
   'Halle Berry',
   'Hugh Jackman',
   'Ian McKellen',
   'Patrick Stewart'): 3}]

# Submission

In [532]:
# в конце можно посмотреть свои ответы к каждому вопросу
#for index,value in sorted(answers.items():
#  print(index,value)
for i in range(1,28):
  answers.setdefault(str(i),'')
  print(str(i).rjust(2),'->',answers[str(i)])

 1 -> 723. Pirates of the Caribbean: On Stranger Tides (tt1298650)
 2 -> 1157. Gods and Generals (tt0279111)
 3 -> 768. Winnie the Pooh (tt1449283)
 4 -> 110
 5 -> 107
 6 -> 239. Avatar (tt0499549)
 7 -> 1245. The Lone Ranger (tt1210819)
 8 -> 1478
 9 -> 599. The Dark Knight (tt0468569)
10 -> 1245. The Lone Ranger (tt1210819)
11 -> Drama
12 -> Drama
13 -> Peter Jackson
14 -> Robert Rodriguez
15 -> Chris Hemsworth
16 -> Matt Damon
17 -> Action
18 -> 925. K-19: The Widowmaker (tt0267626)
19 -> 2015
20 -> 2014
21 -> September
22 -> 450
23 -> Peter Jackson
24 -> Four By Two Productions
25 -> Midnight Picture Show
26 -> ['Inside Out', 'The Dark Knight', '12 Years a Slave']
27 -> ('Daniel Radcliffe', 'Rupert Grint')


In [533]:
# и убедиться что ни чего не пропустил)
len(answers)

27